In [83]:
%matplotlib inline
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

# Adjust display settings
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', 120) # Make DataFrame column with wide enough to support pipedrive link...

In [84]:
# Data: MovieLens _ml
# Read in Movies (m) and Ratings (r) from MovieLens Data
dataDir = '/Users/justinbarton/Documents/Personal/DataScienceIntensive/Data'
mov_ml = pd.read_csv(dataDir + '/MovieLens/ml-20m/movies.csv')
rat_ml = pd.read_csv(dataDir + '/MovieLens/ml-20m/ratings.csv')

In [85]:
# Data: MovieLens _ml
# Find the average movie rating and store in 'mr', remove those without a rating
mov_rat_ml = pd.DataFrame(rat_ml[['movieId','rating']].groupby('movieId').mean())
mov_rat_ml.head()

,rating
movieId,
1,3.921240
2,3.211977
3,3.151040
4,2.861393
5,3.064592


In [86]:
# Data: MovieLens _ml
# Count the number of ratings and store in 'mc'. Sort by highest number of ratings forst
mov_num_rat_ml = pd.DataFrame(rat_ml[['movieId','rating']].groupby('movieId').size(), columns=['num_ratings']).sort_values(by='num_ratings', ascending=0)
mov_num_rat_ml.head()

,num_ratings
movieId,
296,67310
356,66172
318,63366
593,63299
480,59715


In [87]:
# Data: MovieLens
# Show all movies, number of ratings 'num_ratings' and average rating 'rating'
mov_rat_agg_ml = mov_num_rat_ml.merge(mov_rat_ml, right_index=1, left_index=1)
mov_rat_agg_ml.head()

,num_ratings,rating
movieId,,
1,49695,3.921240
2,22243,3.211977
3,12735,3.151040
4,2756,2.861393
5,12161,3.064592


In [88]:
# Data: MovieLens _ml
# Aggregate all movies, and ratings stats into one Dataframe (Note: this removes unrated films)
# Sort by top movies if sorted by average rating, then number of ratings
agg_ml = (mov_ml.merge(mov_rat_agg_ml,how='inner', left_on='movieId',right_index=1).groupby(['title','genres']).mean().
    sort_values(['rating','num_ratings'], ascending=[0,0]))
agg_ml.reset_index(inplace=True)
agg_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26739 entries, 0 to 26738
Data columns (total 5 columns):
title          26739 non-null object
genres         26739 non-null object
movieId        26739 non-null float64
num_ratings    26739 non-null float64
rating         26739 non-null float64
dtypes: float64(3), object(2)
memory usage: 1.0+ MB


In [90]:
# Data: MovieLens
# Separate the year from the title into separate columns for MovieLen titles
df = agg_ml.copy()
df['year'] = df.title.str.extract('[(](\d{4})[)]') 
df['title'] = df.title.str.replace(r' [(]\d{4}[)]', '')
df['year'].fillna('0', inplace=True)
df['year'] = df['year'].astype(int)
agg_sepYr_ml = df
df

,title,genres,movieId,num_ratings,rating,year
0,Catastroika,Documentary,108527.0,2.0,5.0,2012
1,Consuming Kids: The Commercialization of Childhood,Documentary,103871.0,2.0,5.0,2008
2,1971,Documentary,114254.0,1.0,5.0,2014
3,A Blank on the Map,Documentary,129478.0,1.0,5.0,1971
4,A Gun for Jennifer,Crime|Drama|Thriller,129295.0,1.0,5.0,1997
5,A Night for Dying Tigers,Drama,121039.0,1.0,5.0,2010
6,Abendland,Documentary,101292.0,1.0,5.0,2011
7,Afstiros katallilo,Comedy,129243.0,1.0,5.0,2008
8,Al otro lado,Drama,40404.0,1.0,5.0,2004
9,Always for Pleasure,(no genres listed),125599.0,1.0,5.0,1978


In [91]:
# Data: MovieLens
# Move the 'The' to the start of the film title for MovieLens titles to help with matching
# (e.g. "Usual Suspects, The" in MovieLens is "The Usual Suspects" in IMDB & OMDB)
df = agg_sepYr_ml.copy()
df['has_the'] = df.title.str.contains(', The')
df.title = df.title.str.replace(r', The', '')
df.ix[df.has_the, 'title'] = 'The ' + df.ix[df.has_the, 'title']
agg_sepYr_moveThe_ml = df
df[df.has_the].head()

,title,genres,movieId,num_ratings,rating,year,has_the
13,The Barchester Chronicles,Drama,95517.0,1.0,5.0,1982,True
14,The Best of Ernie and Bert,Children,94972.0,1.0,5.0,1988,True
23,The Codes of Gender,Documentary,113860.0,1.0,5.0,2010,True
34,The Foster Brothers (Süt kardesler),Comedy,86055.0,1.0,5.0,1976,True
40,The Human Behavior Experiments,Documentary,103753.0,1.0,5.0,2006,True


In [94]:
# Data: OMDB
# Read in data from from omdb Data
dataDir = '/Users/justinbarton/Documents/Personal/DataScienceIntensive/Data'
mov_om = pd.read_csv(dataDir + '/omdb/all_movies.csv', error_bad_lines=False)
det_om = pd.read_csv(dataDir + '/omdb/movie_details.csv')
lan_om = pd.read_csv(dataDir + '/omdb/movie_languages.csv')
tra_om = pd.read_csv(dataDir + '/omdb/trailers.csv')
ppl_om = pd.read_csv(dataDir + '/omdb/all_people.csv')
job_om = pd.read_csv(dataDir + '/omdb/job_names.csv')
cst_om = pd.read_csv(dataDir + '/omdb/all_casts.csv', error_bad_lines=False)

# Find all jobs for english speaking films
jobs = job_om[job_om.language_iso_639_1=='en'].merge(cst_om, left_on='job_id', right_on='job_id')
#most_common_jobs = jobs.groupby('name').size().sort_values(ascending=0)
#'Actor', 'Director', 'Screenplay', 'Producer', 'Director of Photography', 'Editor', 'Original Music Composer'
#'Editor', 'Original Music Composer', 'Executive Producer', 'Music'

# Find all directors and their movies
directorID = job_om[(job_om.name=='Director')&(job_om.language_iso_639_1=='en')].job_id.iloc[0]
directors_movies = jobs[jobs.name=='Director'].merge(ppl_om, left_on='person_id', right_on='id')[['movie_id','person_id','name_y']]
directors_movies.head()

Skipping line 6074: expected 4 fields, saw 5
Skipping line 28493: expected 4 fields, saw 5

Skipping line 39995: expected 5 fields, saw 6
Skipping line 110384: expected 5 fields, saw 6

Skipping line 187226: expected 5 fields, saw 6
Skipping line 237735: expected 5 fields, saw 9
Skipping line 237736: expected 5 fields, saw 9

Skipping line 332128: expected 5 fields, saw 8
Skipping line 332424: expected 5 fields, saw 7
Skipping line 348703: expected 5 fields, saw 6
Skipping line 365812: expected 5 fields, saw 6

Skipping line 655284: expected 5 fields, saw 6
Skipping line 655285: expected 5 fields, saw 6

Skipping line 679612: expected 5 fields, saw 6



,movie_id,person_id,name_y
0,11,1,George Lucas
1,636,1,George Lucas
2,838,1,George Lucas
3,1893,1,George Lucas
4,1894,1,George Lucas


In [95]:
# Data: OMDB
# Add the Year from the OMDB data from the date
df = mov_om.copy()
df.date.astype(str, inplace=True) # Convert dates to string
df.loc[df.date == '\N'] = '' # Remove null dates '/N'
df['year'] = pd.to_datetime(df.date.astype(str), format='%Y-%m-%d') # Get the date format
df['year'] = df['year'].dt.year.fillna(0).astype(int) # Extract the year as an int, from the date
mov_addYr_om = df
mov_addYr_om.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39726 entries, 0 to 39725
Data columns (total 5 columns):
id           39726 non-null object
name         39726 non-null object
parent_id    39726 non-null object
date         39726 non-null object
year         39726 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [96]:
# Data: MovieLens & OMDB
# Merge the MovieLens and OMDB data into one dataframe called 'films' (merging by movie title and year)
# (This is a very basic way to join movie titles together, but will suffice for proof of concept.
# Some examples of incorrect mismatches to be studied later include:
# 'City of God (Cidade de Deus)' in MovieLens and 'City of God' in OMDB 
# 'One Flew Over the Cuckoo's Nest' in MovieLens and 'One Flew Over The Cuckoo's Nest' in OMDB
# 'Star Wars: Episode IV - A New Hope' in MovieLens and 'Star Wars: Episode IV – A New Hope' in OMDB)
df = agg_sepYr_moveThe_ml.merge(mov_addYr_om, left_on=['title', 'year'], right_on=['name', 'year'], how='left')
df = df[df.name.notnull()]
films_ml_om = df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10266 entries, 6 to 26774
Data columns (total 11 columns):
title          10266 non-null object
genres         10266 non-null object
movieId        10266 non-null float64
num_ratings    10266 non-null float64
rating         10266 non-null float64
year           10266 non-null int64
has_the        10266 non-null bool
id             10266 non-null object
name           10266 non-null object
parent_id      10266 non-null object
date           10266 non-null object
dtypes: bool(1), float64(3), int64(1), object(6)
memory usage: 892.3+ KB


In [98]:
# Data: IMDB
# Read in IMDB Data
cst_im = pd.read_csv(dataDir +'/IMDB/cast.csv')
mov_im = pd.read_csv(dataDir +'/IMDB/titles.csv')

# Create data frame with titles and films, it and with id 'index'
mov_im.reset_index(inplace=True)
mov_cst_im = cst_im.merge(mov_im)

mov_cst_im.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3475509 entries, 0 to 3475508
Data columns (total 7 columns):
title        object
year         int64
name         object
type         object
character    object
n            float64
index        int64
dtypes: float64(1), int64(2), object(4)
memory usage: 212.1+ MB


In [100]:
# Data: MovieLens & OMDB & IMDB
# Connect IMDB data to 'films' in order to call upon cast-list
# (similarly to above, 'title' and 'year' here do not provide a full-proof matching system between data sets,
# will need to analyse this further in future versions.. For example:
# 'V for Vendetta' is a 2006 movie in MovieLens and OMDB, but its a 2005 movie in IMDB,
# 'Batman: The Dark Knight Returns, Part 2' for MovieLens/OMDB and 'The Dark Knight: Knightfall - Part Two' in IMDB
# 'Birdman' in MovieLens and OMDB and 'Birdman or (The Unexpected Virtue of Ignorance)' in IMDB
df = films_ml_om
imdb_connector_fields = ['title', 'year']
df = df.merge((mov_cst_im[['index']+imdb_connector_fields]).drop_duplicates(), left_on=['title', 'year'], 
              right_on = imdb_connector_fields, how='inner')
films_all = df

# Add the lead actor to the list (this will help new reviewers rating films to identify films)
df = mov_cst_im.copy()
df = df[df.n == 1][['index','name']].drop_duplicates()
df = df.merge(films_all, left_on='index',right_on='index', how='inner')
df.rename(columns={'name_x':'lead actor','name_y':'name'}, inplace=True)
films_all = df
films_all.info()

# NOTE:
# films_all - represents all of the films that can be used as recommendtaions.
# This is the current best intersection of 3 datasets: MovieLens, OMDB, IMDB.
# As the matching algorithms between these sets improve, this number will increase

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8832 entries, 0 to 8831
Data columns (total 13 columns):
index          8832 non-null int64
lead actor     8832 non-null object
title          8832 non-null object
genres         8832 non-null object
movieId        8832 non-null float64
num_ratings    8832 non-null float64
rating         8832 non-null float64
year           8832 non-null int64
has_the        8832 non-null bool
id             8832 non-null object
name           8832 non-null object
parent_id      8832 non-null object
date           8832 non-null object
dtypes: bool(1), float64(3), int64(2), object(7)
memory usage: 905.6+ KB


In [101]:
# Create dataframe of films that users can review...

# Remove films without an english language
df = films_all
df = df.merge(lan_om, left_on='id', right_on='movie_id')
df = df[df.language_iso_639_1 == 'en']

# Remove films made before 1965
df = df[df.year >= 1965]

# Remove films with < 10 number of ratings
df = df[df.num_ratings >= 10]

# Get rid of duplicates (since some films have mulitple lead-actor (n=1) and/or genre)
df = df.drop_duplicates(subset=['index'])

# Sort by best films at the start
df = df.sort_values(by=['rating', 'num_ratings'], ascending=[False, False])

films_reviewable = df
df


,index,lead actor,title,genres,movieId,num_ratings,rating,year,has_the,id,name,parent_id,date,movie_id,language_iso_639_1
4270,129143,Tim Robbins,The Shawshank Redemption,Crime|Drama,318.0,63366.0,4.446990,1994,True,278,The Shawshank Redemption,\N,1994-09-10,278,en
4133,107741,Marlon Brando,The Godfather,Crime|Drama,858.0,41355.0,4.364732,1972,True,238,The Godfather,230,1972-03-15,238,en
6803,5601,Stephen Baldwin,The Usual Suspects,Crime|Mystery|Thriller,50.0,47006.0,4.334372,1995,True,629,The Usual Suspects,\N,1995-08-16,629,en
5061,220907,Liam Neeson,Schindler's List,Drama|War,527.0,50054.0,4.310175,1993,False,424,Schindler's List,\N,1993-11-30,424,en
4584,32984,Edward Norton,Fight Club,Action|Crime|Drama|Thriller,2959.0,40106.0,4.227123,1999,False,550,Fight Club,\N,1999-09-10,550,en
2249,9629,Christian Bale,The Dark Knight,Action|Crime|Drama|IMAX,58559.0,20438.0,4.220129,2008,True,155,The Dark Knight,263,2008-07-18,155,en
5541,65805,Jack Nicholson,Chinatown,Crime|Film-Noir|Mystery|Thriller,1252.0,15310.0,4.199673,1974,False,829,Chinatown,\N,1974-06-20,829,en
4229,150352,Mark Hamill,Star Wars: Episode V - The Empire Strikes Back,Action|Adventure|Sci-Fi,1196.0,45313.0,4.188202,1980,False,1891,Star Wars: Episode V - The Empire Strikes Back,10,1980-05-21,1891,en
5120,132870,Keanu Reeves,The Matrix,Action|Sci-Fi|Thriller,2571.0,51334.0,4.187186,1999,True,603,The Matrix,2344,1999-03-31,603,en
1680,95224,Robert De Niro,Goodfellas,Crime|Drama,1213.0,26406.0,4.183633,1990,False,769,Goodfellas,\N,1990-09-19,769,en


In [102]:
# Randomise DataFrame, by placing the top 500 films in random order, then the remaining films in random order
# This is in an attempt to reduce a bias, that people will typically rate more prestigious films higher 
# as they would appear that way in the list otherwise. For example Schindler's List, The Godfather etc are well known to rate highly
df = films_reviewable.head(1000).copy()
top500 = df.head(500)
top500 = top500.reindex(pd.np.random.permutation(top500.index))
remainder = df.tail(500)
remainder = remainder.reindex(pd.np.random.permutation(remainder.index))
films_reviewable_randomised = top500.append(remainder)
films_reviewable_randomised.head()

# Output to Excel to then be imported manually into Google Sheets for new user to review
#(is quicker than writing directly to google sheets from pandas)
films_reviewable_randomised[['title','year','lead actor','genres','index']].to_excel('output.xlsx')


In [120]:
%%time
# Dynamically read new user ratings from various google sheets

# Adapted from merging code between: 
# http://gspread.readthedocs.io/en/latest/oauth2.html - Setting up the authentication
# http://pbpython.com/pandas-google-forms-part1.html - Structure of code (with different 'credentials' var to above)
# https://github.com/burnash/gspread - Reading and manipulating the google sheet

from __future__ import print_function
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json

# Authenication and Google Sheet Parameters & Variables
SCOPE = ["https://spreadsheets.google.com/feeds"]
SECRETS_FILE = "/Users/justinbarton/Documents/DevSetup/Justin-ee2a176f3e01.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(SECRETS_FILE, SCOPE)

# Authorise the Google Sheet to open it.
gc = gspread.authorize(credentials)

# Define variables to open spreadsheets and hold new user ratings
users = {}

# Read new user ratings
users['Films_Barton'] = {}
read_new_user_ratings('Films_Barton', 'Damian')
read_new_user_ratings('Films_Barton', 'Jess')

In [119]:
# Function to read in and clean-up new user ratings data
def read_new_user_ratings(workbook_name, sheet_name):
    # Read in data
    workbook = gc.open(workbook_name)
    df = pd.DataFrame(workbook.worksheet(sheet_name).get_all_records())
    
    # Convert 'out of 10' column to correct data
    df.loc[(df['out of 10']=='')|(df['out of 10']=='-'),'out of 10'] = -1
    df['out of 10'] = df['out of 10'].astype(int)
    
    # Add to dictionary
    users[workbook_name][sheet_name] = df

In [106]:
# From here on is a work in progress...

In [121]:
df = users['Films_Barton']['Damian'].append(users['Films_Barton']['Jess'])
df

,genres,index,lead actor,out of 10,title,year
0,Comedy|Crime,150538,Woody Allen,-1,Take the Money and Run,1969
1,Adventure|Animation|Children|Comedy|Fantasy,188190,Tom Hanks,6,Toy Story,1995
2,Drama,140753,Denzel Washington,6,Remember the Titans,2000
3,Action|Adventure|Sci-Fi,212527,Mark Hamill,8,Star Wars: Episode VI - Return of the Jedi,1983
4,Drama|Mystery|Thriller,153955,Nicole Kidman,7,Dogville,2003
5,Western,214479,Clint Eastwood,-1,High Plains Drifter,1973
6,Crime|Drama|Mystery,197667,Josh Hartnett,-1,Lucky Number Slevin,2006
7,Drama,201168,Jesse Eisenberg,8,The Social Network,2010
8,Drama,195150,Brad Pitt,8,Moneyball,2011
9,Comedy|Mystery|Thriller,43821,Laurence Olivier,-1,Sleuth,1972


In [137]:
# Combine to users together for directors report.
df1 = users['Films_Barton']['Damian']
df1 = df1[df1['out of 10'] != -1]
df2 = users['Films_Barton']['Jess']
df2 = df2[df2['out of 10'] != -1]
df = pd.merge(df1, df2)
len(df1), len(df2), len(df)

(263, 107, 26)

In [138]:
# Find users preferred directors
#df = jb_films
#df = users['Films_Barton']['Damian'].merge(users['Films_Barton']['Jess'])

df = df.merge(films_all[['id','index']].drop_duplicates(), left_on='index', right_on='index').drop_duplicates()
users_directors = df[df['out of 10'] >= 7].merge(directors_movies, left_on='id', right_on='movie_id')
director_counts = pd.DataFrame(users_directors.groupby(['name_y']).size(), columns=['director_count'])
users_directors['director_count'] = users_directors.merge(director_counts, left_on='name_y', right_index=True)['director_count']

fields_desired = ['out of 10','title', 'year','name_y', 'genres','lead actor']

df = users_directors.sort_values(by=['director_count','name_y', 'out of 10'], ascending=[False, True, True])[fields_desired]
df.rename(columns={'name_y':'director'}, inplace=True)
df.drop_duplicates(subset=['title', 'year'], inplace=True)
df['user'] = 'Jemian'

import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
df.to_excel('results.xlsx')

In [ ]:
# Take the above and create a generic function 'my preferred people' tables, which could be directors, screenplay, music, billed actors

In [ ]:
# Find preferred screenplay

In [ ]:
# Find preferred actors/actresses (that are at least credited in that film)

In [ ]:
# Find simliar users

In [ ]:
# OLD CODE - INSERT BELOW

In [ ]:
# To be used for generating the surveys for people
#df = top_better

#top500 = df.head(500)
#top500 = top500.reindex(pd.np.random.permutation(top500.index))
#next4500 = df.head(5000).tail(4500)
#next4500 = next4500.reindex(pd.np.random.permutation(next4500.index))
#output = top500.append(next4500)
#output.head()

In [ ]:
#import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')

# Save output to spreadsheet for each person
#output[['movieId','title']].to_excel('output.xlsx','Sheet1')

In [ ]:
# # Data: Excel - My Movie Ratings
# # Read in the films that I've already reviewed
# pd.set_option('display.max_rows', 5000)
# jb = pd.read_excel('output_JB1.xlsx')
# jb.movieId = jb.movieId.astype(int)
# jb = jb.merge(agg_ml, left_on='movieId', right_on='movieId')
# jb['reviewed'] = jb['out_of_10'].bfill().notnull() # Flag all of the films I've reviwed, so that I don't have to do it again
# jb = jb[jb['reviewed']]
# jb_watched = jb[jb['out_of_10'].notnull()]
# #jb.info()


# # Read in the films that I've already reviewed
# jb2 = pd.read_excel('JB_films2.xlsx')
# jb2 = jb2[['out of 10', 'index']]
# jb2.index = jb2.index.astype(int)
# jb2 = jb2.merge(films_all, left_on='index', right_on='index')
# jb2['reviewed'] = jb2['out of 10'].bfill().notnull() # Flag all of the films I've reviwed, so that I don't have to do it again
# jb2 = jb2[jb2['reviewed']]
# jb2_watched = jb2[jb2['out of 10'].notnull()]
# jb2.rename(columns={'out of 10':'out_of_10'}, inplace=True)
# jb2

# jb = jb.append(jb2)
# jb = jb[['id','index','movieId','out_of_10']]
# jb.drop_duplicates(inplace=True)
# jb.info()

# import sys  
# reload(sys)  
# sys.setdefaultencoding('utf8')
# jb.to_excel('JB_Reviewed_Films.xlsx','Sheet1', na_rep='')

In [ ]:
# jb.out_of_10.fillna(-1,inplace=True)
# jb = jb[jb['index'].notnull()]

In [ ]:
# jb['index'] = jb['index'].astype(int)
# jb['out_of_10'] = jb['out_of_10'].astype(int)
# jb = jb[['index','out_of_10']]
# jb.merge(films_reviewable_randomised, left_on='index', right_on='index', how='right')[['out_of_10','title','year','lead actor','genres','index']].to_excel('output.xlsx','Sheet1')